In [1]:
# Import dependencies
import pandas as pd
import sklearn as skl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import tensorflow as tf

# Read in data
olympic_df = pd.read_csv("./cleandatabmi.csv")
olympic_df.head()

,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,BMI
0,A Dijiang,M,24,180,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,1,25
1,A Lamusi,M,23,170,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,1,21
2,Christine Jacoba Aaftink,F,21,185,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,1,24
3,Christine Jacoba Aaftink,F,21,185,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,"Speed Skating Women's 1,000 metres",1,24
4,Christine Jacoba Aaftink,F,25,185,82.0,Netherlands,NED,1992 Winter,1992,Winter,Albertville,Speed Skating,Speed Skating Women's 500 metres,1,24


In [2]:
#Drop non-feature columns
olympic_df.drop(['Name',"Sex", "Height", "Weight", "Team", "NOC", "Games","Year", "Season","City","Sport", "Event"], inplace=True, axis=1)
olympic_df.head()

,Age,Medal,BMI
0,24,1,25
1,23,1,21
2,21,1,24
3,21,1,24
4,25,1,24


In [3]:
#check unique value count for encoding
olympic_df.nunique(axis=0)

Age      61
Medal     2
BMI      46
dtype: int64

In [4]:
#create categorical variable list for encoding
olympic_cat = olympic_df.dtypes[olympic_df.dtypes == "object"].index.tolist()
olympic_cat

[]

In [5]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(olympic_df[olympic_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(olympic_cat)


In [6]:
#Merge encoded values and numbered medals and drop original columns
olympic_df = olympic_df.merge(encode_df,left_index=True, right_index=True)
olympic_df = olympic_df.drop(olympic_cat,1)

olympic_df.head()

,Age,Medal,BMI
0,24,1,25
1,23,1,21
2,21,1,24
3,21,1,24
4,25,1,24


In [7]:
# Split preprocessed data into features and target arrays
y = olympic_df["Medal"].values
X = olympic_df.drop(["Medal"],1).values

# Split the preprocessed data into training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [8]:
# Create StandardScaler instance
scaler = StandardScaler()

# Fit StandardScaler
X_scaler = scaler.fit(X_train)

# Scale data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define model
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 8)                 24        
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 45        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 6         
Total params: 75
Trainable params: 75
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
4832/4832 [==============================] - 6s 1ms/step - loss: -2.4109 - accuracy: 0.8520
Epoch 2/100
4832/4832 [==============================] - 6s 1ms/step - loss: -6.3644 - accuracy: 0.8547
Epoch 3/100
4832/4832 [==============================] - 5s 1ms/step - loss: -10.2517 - accuracy: 0.8547
Epoch 4/100
4832/4832 [==============================] - 5s 1ms/step - loss: -14.1018 - accuracy: 0.8547
Epoch 5/100
4832/4832 [==============================] - 6s 1ms/step - loss: -17.9821 - accuracy: 0.8547
Epoch 6/100
4832/4832 [==============================] - 6s 1ms/step - loss: -21.8543 - accuracy: 0.8547
Epoch 7/100
4832/4832 [==============================] - 6s 1ms/step - loss: -25.7404 - accuracy: 0.8547
Epoch 8/100
4832/4832 [==============================] - 6s 1ms/step - loss: -29.6118 - accuracy: 0.8547
Epoch 9/100
4832/4832 [==============================] - 7s 1ms/step - loss: -33.4748 - accuracy: 0.8547
Epoch 10/100
4832/4832 [==============================] -

4832/4832 [==============================] - 5s 1ms/step - loss: -296.8783 - accuracy: 0.8547
Epoch 78/100
4832/4832 [==============================] - 5s 1ms/step - loss: -300.7583 - accuracy: 0.8547
Epoch 79/100
4832/4832 [==============================] - 6s 1ms/step - loss: -304.6393 - accuracy: 0.8547
Epoch 80/100
4832/4832 [==============================] - 10s 2ms/step - loss: -308.5010 - accuracy: 0.8547TA: 2s  - ETA: 0s - loss: -308.5337 - accuracy: 
Epoch 81/100
4832/4832 [==============================] - 7s 1ms/step - loss: -312.3810 - accuracy: 0.8547
Epoch 82/100
4832/4832 [==============================] - 6s 1ms/step - loss: -316.2518 - accuracy: 0.8547A: 5 - ETA: 3s - lo
Epoch 83/100
4832/4832 [==============================] - 6s 1ms/step - loss: -320.1350 - accuracy: 0.8547A: 6s - ETA: 1s - loss
Epoch 84/100
4832/4832 [==============================] - 6s 1ms/step - loss: -324.0089 - accuracy: 0.8547
Epoch 85/100
4832/4832 [==============================] - 6s 1ms/st

In [16]:
# Evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

1611/1611 - 2s - loss: -3.9907e+02 - accuracy: 0.8505
Loss: -399.0748596191406, Accuracy: 0.8504714369773865
